In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import rc
import time
path = "/home/esteban/Escritorio/Practica/Codes/C_Python"
os.chdir( path )
import SPack as SP
rc('text', usetex=True)
rc('font', family='serif')

In [ ]:
%%time
rhos     = [3.5, 3.0, 2.5, 2.0]
dc       = [1,2,3,4]               # Density cuts
models = ['G13','GP14', 'L16']
type_cut = ['Stellar Mass/', 'SFR/']
NBIN, lmax, lmin = 30, 15, 10.
bin_width = ( lmax - lmin ) / NBIN

for model in models:
    
    ab = model
    model = ab + "_millennium"
    path = "/home/esteban/Escritorio/Practica/Data/" + model
    os.chdir( path )
    
    # Loading
    x = np.load( 'x.npy' )
    y = np.load( 'y.npy' )
    z = np.load( 'z.npy' )

    if ab == 'G13': halomass = np.log10( np.load( 'centralmvir.npy' ) + 1e-10 ) + 10
    else: halomass = np.log10( np.load( 'mdhalo.npy' ) + 1e-10 ) + 10
    type_gal = np.load( 'type.npy' )

    idx = np.where( type_gal == 0 )[0] # Positions of the centrals

    # Centrals Data
    x_cen = x[idx]
    y_cen = y[idx]
    z_cen = z[idx]
    halomass_cen = halomass[idx]
    
    del x, y, z, halomass
    for cut in type_cut:
        
        hod_path = path + '/hod_sams/' + cut
        cat_path = path + '/Mock Catalogues/' + cut
        os.chdir( cat_path )
        
        for c, rho in zip(dc, rhos):
            print c, rho
    
            # Loading HOD data
            
            bins, HOD, Cent, Sats = np.loadtxt( hod_path + 'hod_d%i_30b' %c, unpack = True )

            # Straight parameters equation
            params = np.zeros((NBIN - 1, 2), dtype = np.float32)
            for i in range( NBIN-1 ):
                y1, y2 = HOD[i], HOD[i+1]
                x1, x2 = bins[i], bins[i+1]
                m = ( y2 - y1 ) / ( x2 - x1)
                n = y1 - (m * x1)
                params[i] = np.array([m, n])

            mask = (halomass_cen >= bins[0]) & (halomass_cen <= bins[-1])
            halomass_cen = halomass_cen[mask]
            x_cen, y_cen, z_cen = x_cen[mask], y_cen[mask], z_cen[mask]
            Len_Cat = len(x_cen)
            
            #%%time
            mass_index = ((( halomass_cen - bins[0] ) / ( bins[-1] - bins[0] )) * (NBIN-1)).astype('int32')
            n_gal = np.zeros(Len_Cat).astype('float64')
            #print "todo ok"
            
            M, N = params[:,0], params[:,1]
            M = M.copy(order='C')
            N = N.copy(order='C')
            SP.Ngals(n_gal, halomass_cen, mass_index, M, N, Len_Cat)
        
            #for i in xrange(len(halomass_cen)):
            #    index = mass_index[i]
            #    m, n = params[index]
            #   log_n_gal = m * halomass_cen[i] + n
            #    n_gal[i] = 10**(log_n_gal)   

            #%%time
            nn_gal = np.zeros(Len_Cat)
            for i in xrange(Len_Cat):
                n = n_gal[i]
                dec = n % 1
                rr = np.random.random_sample()
                if rr <= dec: nn_gal[i] = int(n) + 1
                else: nn_gal[i] = int(n)    
            
            del n_gal
            #%%time
            
            #j = 0
            #f = open('cat_d%i_fixed.txt' %(k+1), 'w')
            
            cat_data = np.zeros((Len_Cat,5), dtype = np.float32)
            #print "Len Haloes: ", len(x_cen)
        
            for i,n in enumerate(nn_gal):
                cat_data[i] = np.array([x_cen[i], y_cen[i], z_cen[i], halomass_cen[i], n])
            
            
            np.save('cat_d%i_fixed' %c, cat_data)  
            
        
            print "Model: ", model, "Cut: ", cut, "Density: ", c
   

In [ ]:
%%time
rhos     = [2.0, 2.5, 3.0, 3.5]
dc       = [4,3,2,1]                # Density cuts
#models = ['G13']
#type_cut = ['Stellar Mass/']
models   = ['G13','GP14','L16']
type_cut = ['Stellar Mass/', 'SFR/']

NBIN, lmax, lmin = 30, 15, 10.
bin_width = ( lmax - lmin ) / NBIN

for model in models:
    
    ab = model
    model = ab + "_millennium"
    path = "/home/esteban/Escritorio/Practica/Data/" + model
    os.chdir( path )
    
    # Loading
    x = np.load( 'x.npy' )
    y = np.load( 'y.npy' )
    z = np.load( 'z.npy' )
    if ab == 'G13': halomass = np.log10( np.load( 'centralmvir.npy' ) + 1e-10 ) + 10
    else: halomass = np.log10( np.load( 'mdhalo.npy' ) + 1e-10 ) + 10
    type_gal = np.load( 'type.npy' )

    idx = np.where( type_gal == 0 )[0] # Positions of the centrals

    # Centrals Data
    x_cen = x[idx]
    y_cen = y[idx]
    z_cen = z[idx]
    halomass_cen = halomass[idx]
    
    del x, y, z, halomass
    for cut in type_cut:
        hod_path = path + '/hod_sams/' + cut
        cat_path = path + '/Mock Catalogues/' + cut
        os.chdir( cat_path )

        for c, rho in zip(dc, rhos):
    
            # Loading HOD data
            bins, HOD, Cent, Sats = np.loadtxt( hod_path + 'hod_d%i_30b' %c, unpack = True )
            #bins = bins - bin_width * 0.5 # Move to original position

            # Straight parameters equation
            m_all, n_all = np.zeros(NBIN - 1).astype('float32'), np.zeros(NBIN - 1).astype('float32')
            m_cen, n_cen = np.zeros(NBIN - 1).astype('float32'), np.zeros(NBIN - 1).astype('float32')
            m_sat, n_sat = np.zeros(NBIN - 1).astype('float32'), np.zeros(NBIN - 1).astype('float32')
    
            for i in range( NBIN-1 ):
                y1_all, y2_all = HOD[i], HOD[i+1]
                y1_cen, y2_cen = Cent[i], Cent[i+1]
                y1_sat, y2_sat = Sats[i], Sats[i+1]
                x1, x2 = bins[i], bins[i+1]
            
                m_all[i] = ( y2_all - y1_all ) / ( x2 - x1)
                m_cen[i] = ( y2_cen - y1_cen ) / ( x2 - x1)
                m_sat[i] = ( y2_sat - y1_sat ) / ( x2 - x1)
       
                n_all[i] = y1_all - (m_all[i] * x1)
                n_cen[i] = y1_cen - (m_cen[i] * x1)
                n_sat[i] = y1_sat - (m_sat[i] * x1)
    
            mask = (halomass_cen >= bins[0]) & (halomass_cen <= bins[-1])
            halomass_cen = halomass_cen[mask]
            x_cen, y_cen, z_cen = x_cen[mask], y_cen[mask], z_cen[mask]
            Len_Haloes = len(x_cen)
            
            #%%time
            mass_index = ((( halomass_cen - bins[0] ) / ( bins[-1] - bins[0] )) * (NBIN-1)).astype('int32')
            
            n_gal_all = np.zeros(Len_Haloes).astype('float64')
            n_gal_cen = np.zeros(Len_Haloes).astype('float64')
            n_gal_sat = np.zeros(Len_Haloes).astype('float64')
        
            SP.Ngals2(n_gal_all, n_gal_cen, n_gal_sat, halomass_cen, mass_index, m_all, n_all, m_cen, n_cen,
                     m_sat, n_sat, Len_Haloes)

            #nn_gal = np.zeros(Len_Cat)
            for i in xrange(Len_Haloes):
                n1 = n_gal_all[i]
                n2 = n_gal_cen[i]
                n3 = n_gal_sat[i]
                dec1 = n1 % 1
                dec2 = n2 % 1
                dec3 = n3 % 1
                rr1 = np.random.random_sample()
                rr2 = np.random.random_sample()
                rr3 = np.random.random_sample()
                if rr1 <= dec1: n_gal_all[i] = int(n1) + 1
                else: n_gal_all[i] = int(n1)      
                if rr2 <= dec2: n_gal_cen[i] = int(n2) + 1
                else: n_gal_cen[i] = int(n2)      
                if rr3 <= dec3: n_gal_sat[i] = int(n3) + 1
                else: n_gal_sat[i] = int(n3)    
            
            cat_data = np.zeros((Len_Haloes,7), dtype = np.float32)
        
            for i in range(Len_Haloes):
                cat_data[i] = np.array([x_cen[i], y_cen[i], z_cen[i], halomass_cen[i], n_gal_all[i], n_gal_cen[i], n_gal_sat[i]])
        
            np.save('cat_d%i_2HOD' %c, cat_data)  
            
            print "termine density: ", rho 
        print "termine cut: ", cut
    print "termine model: ", model 


In [ ]:
logm_min = 10
logm_max = 14.8333333
NBIN = 29
bin_width = (logm_max - logm_min)/NBIN

#models = ['G13', 'GP14', 'L16']
#tc = ['Stellar Mass/', 'SFR/']
#rhos = [2.0, 2.5, 3.0, 3.5]
#dc = [4, 3, 2, 1]


models = ['G13']
tc = ['Stellar Mass/']
rhos = [3.0]
dc = [2]

for model in models:
    
    ab = model
    path = "/home/esteban/Escritorio/Practica/Data/" + ab + "_millennium"
    path_plots = "/home/esteban/Escritorio/Practica/Plots/" + ab + "_millennium"
    
    for cut in tc:
   
        cat_path = path + '/Mock Catalogues/'+ cut
        hod_path = path + '/hod_data/' + cut
        hod_plot_path = path_plots + '/HOD_cat/' + cut
        
        for c, rho in zip(dc, rhos):

            data = np.load(cat_path + 'cat_d%i_fixed.npy' %c)
            sam_data = np.loadtxt(hod_path + 'hod_%.1f' %rho)
            bins_sam, hod_sam = sam_data[:,0] , sam_data[:,1]
            x,y,z,halomass,n = data[:,0], data[:,1], data[:,2], data[:,3], data[:,4]

            mass_labels = ((halomass - logm_min)/(logm_max - logm_min) * NBIN).astype(int)
            bin_width = (logm_max - logm_min)/NBIN
            bins_p = np.array([logm_min + bin_width*i for i in range(NBIN)]) + bin_width * 0.5

            HOD = np.zeros(NBIN)
            for i in range(NBIN):

                idx2 = np.where(mass_labels == i)[0]  # numero de halos
                idx1 = np.where((mass_labels == i) & (n != 0))[0]
                N_mask = n[idx1]
                Len_Haloes = len(idx2) + 1e-10
                Len_Full = np.sum(N_mask)
                HOD[i]      = Len_Full / Len_Haloes 

            HOD = np.log10(HOD)
            Len_cat = np.sum(n)
            d_cat = np.log10(Len_cat/(500.**3))
            
            f = plt.figure()
            plt.axhline(y = 0, color = 'black', linestyle = '--', lw = .4)
            plt.plot(bins_sam, hod_sam, 'k.', markersize = 3.5, label = r"All / SAM -- $n = 10^{-%.1f} /h^{-3} Mpc^3$" %rho)
            plt.plot(bins_p, HOD, 'k--', markersize = 1.5, label = r"All / Mock -- $n = 10^{%.1f} /h^{-3} Mpc^3$" %d_cat)
            #plt.title(r"$n = 10^{-%.1f} /h^{-3} Mpc^3$" %rho)
            plt.xlabel(r"$log(M_h / h^{-1}M_{\odot})$", fontsize = 10)
            plt.ylabel(r"$log(N)$", fontsize = 10)
            plt.ylim(-2,3)
            plt.xlim(10,15)
            plt.minorticks_on()
            plt.legend(loc='upper left',title = r"\textbf{%s}"% ab, frameon = False, prop={'size':10})
            f.savefig(hod_plot_path + 'hod_%.1f.pdf' %rho)